In [2]:
import mpmath as mp
from mpmath import *
import numpy as np
import pandas as pd
import time

mp.dps = 500
mp.prec = 166

lnMpinGev = mp.mpf('42.334')
Mp = mp.exp(lnMpinGev)

mphi_tree = lambda phi, phigut, mphigut : mphigut
lambda6_tree = lambda phi, phigut, lambda6gut : lambda6gut
A6_tree = lambda phi, phigut, A6gut, mphigut : A6gut

g1gut, g2gut, g3gut = mp.sqrt(mp.pi/mp.mpf('6')), mp.sqrt(mp.pi/mp.mpf('6')), mp.sqrt(mp.pi/mp.mpf('6'))
pre = mp.mpf('1')/(mp.mpf('8')*mp.pi**2)
b1, b2, b3 = mp.mpf('33')/mp.mpf('5'), mp.mpf('1'), mp.mpf('-3')
g1 = lambda phi, phigut : g1gut/(mp.sqrt(1-pre*b1*g1gut**2*mp.log(phi/phigut)))
g2 = lambda phi, phigut : g2gut/(mp.sqrt(1-pre*b2*g2gut**2*mp.log(phi/phigut)))
g3 = lambda phi, phigut : g3gut/(mp.sqrt(1-pre*b3*g3gut**2*mp.log(phi/phigut)))

xi = mp.mpf('1') # éventuellement le passer en argument
m1gut = lambda mphigut : xi*mphigut
m2gut = lambda mphigut : xi*mphigut
m3gut = lambda mphigut : xi*mphigut
m1 = lambda phi, phigut, mphigut : m1gut(mphigut)*(g1(phi, phigut)/g1gut)**mp.mpf('2')
m2 = lambda phi, phigut, mphigut : m2gut(mphigut)*(g2(phi, phigut)/g2gut)**mp.mpf('2')
m3 = lambda phi, phigut, mphigut : m3gut(mphigut)*(g3(phi, phigut)/g3gut)**mp.mpf('2')

mphi_lle = lambda phi, phigut, mphigut : mp.sqrt(mphigut**2+(m2gut(mphigut)**2-m2(phi, phigut, mphigut)**2)+mp.mpf('1')/11*(m1gut(mphigut)**2-m1(phi, phigut,mphigut)**2))
A6_lle = lambda phi, phigut, A6gut, mphigut : A6gut-mp.mpf('6')*(m2gut(mphigut)-m2(phi, phigut, mphigut))-mp.mpf('6')/11*(m1gut(mphigut)-m1(phi, phigut,mphigut))
lambda6_lle = lambda phi, phigut, lambda6gut : lambda6gut*(g2gut/g2(phi, phigut))**mp.mpf('6')*(g1gut/g1(phi, phigut))**(mp.mpf('6')/11)
mphi_udd = lambda phi, phigut, mphigut : mp.sqrt(mphigut**2-mp.mpf('8')/9*(m3gut(mphigut)**2-m3(phi, phigut, mphigut)**2)+mp.mpf('4')/99*(m1gut(mphigut)**2-m1(phi, phigut, mphigut)**2))
A6_udd = lambda phi, phigut, A6gut, mphigut : A6gut+mp.mpf('16')/3*(m3gut(mphigut)-m3(phi, phigut, mphigut))-mp.mpf('8')/33*(m1gut(mphigut)-m1(phi, phigut, mphigut))
lambda6_udd = lambda phi, phigut, lambda6gut : lambda6gut*(g3gut/g3(phi, phigut))**(mp.mpf('-16')/3)*(g1gut/g1(phi, phigut))**(mp.mpf('8')/33)

def V_MSSM(phi, infl_type, mphigut, A6gut, lambda6gut):
    phigut = mp.mpf('3e16')
    if infl_type == 0 or infl_type == 'tree':
        mphi_func, A6_func, lambda6_func = mphi_tree, A6_tree, lambda6_tree
    elif infl_type == 1 or infl_type == 'lle':
        mphi_func, A6_func, lambda6_func = mphi_lle, A6_lle, lambda6_lle
    elif infl_type == 2 or infl_type == 'udd':
        mphi_func, A6_func, lambda6_func = mphi_udd, A6_udd, lambda6_udd
    else:
        return 'Error: unknown type of inflation'
    lambda6 = lambda6_func(phi, phigut, lambda6gut)
    mphi = mphi_func(phi, phigut,mphigut)
    A6 = A6_func(phi, phigut, A6gut, mphigut)
    V = mp.mpf('0.5')*mphi**mp.mpf('2')*phi**mp.mpf('2')-lambda6*A6/(mp.mpf('6')*Mp**mp.mpf('3'))*phi**mp.mpf('6')+lambda6**mp.mpf('2')*phi**mp.mpf('10')/Mp**mp.mpf('6')
    return V

def norm_potential(x,alpha,phi0):
    return x**mp.mpf('2')-mp.mpf('2')/3*alpha*x**6+alpha/mp.mpf('5')*x**10

def norm_eps1(x,alpha,phi0):
    return mp.mpf('450')*(mp.mpf('1')+alpha*x**4*(-mp.mpf('2')+x**4))**2/((phi0*x)**mp.mpf('2')*(mp.mpf('15')+alpha*x**4*(-mp.mpf('10')+mp.mpf('3')*x**4))**2)

def norm_eps2(x,alpha,phi0):
    return (mp.mpf('60')*(mp.mpf('15')+alpha*x**mp.mpf('4')*(mp.mpf('40')+x**mp.mpf('4')*(-mp.mpf('78')+alpha*(mp.mpf('20')+mp.mpf('3')*x**8)))))/(phi0**2*x**2*(mp.mpf('15')+alpha*x**mp.mpf('4')*(-mp.mpf('10')+mp.mpf('3')*x**4))**2)
       
def norm_eps3(x,alpha,phi0):
    return (mp.mpf('60')*(mp.mpf('1')+alpha*x**4*(-mp.mpf('2')+x**mp.mpf('4')))*(mp.mpf('225')+alpha*x**mp.mpf('4')*(-mp.mpf('1350')+x**4*(mp.mpf('3915')+alpha*(-mp.mpf('2100')+mp.mpf('20')*(mp.mpf('81')-mp.mpf('10')*alpha)*x**4+mp.mpf('15')*(-mp.mpf('99')+mp.mpf('20')*alpha)*x**mp.mpf('8')+mp.mpf('90')*alpha*x**mp.mpf('12')+mp.mpf('9')*alpha*x**16)))))/(x**2*(mp.mpf('15')+alpha*x**mp.mpf('4')*(-mp.mpf('10')+mp.mpf('3')*x**4))**2*(mp.mpf('15')+alpha*x**4*(mp.mpf('40')+x**4*(-mp.mpf('78')+alpha*(mp.mpf('20')+mp.mpf('3')*x**8)))))/phi0**2

def x_endinf(alpha,phi0):
    xstart = mp.mpf('0.9841521')
    return mp.findroot(lambda x : norm_eps1(x,alpha,phi0)-mp.mpf('1'), xstart,solver='halley', verbose = False)

def efold_primitive(x,alpha,phi0):
    aplus=-alpha+mp.sqrt((alpha**2-alpha)*mp.mpc('1','0'))
    aminus=-alpha-mp.sqrt((alpha**2-alpha)*mp.mpc('1','0'))
    bplus=mp.mpf('2')*(aplus+alpha/mp.mpf('3'))/(aplus-aminus)
    bminus=mp.mpf('2')*(aminus+alpha/mp.mpf('3'))/(aminus-aplus)
    return phi0**2*(mp.re(x**2/20+bplus/(10*mp.sqrt(aplus))*mp.atan(mp.sqrt(aplus)*x**2)+bminus/(10*mp.sqrt(aminus))*mp.atan(mp.sqrt(aminus)*x**2)))

def get_calfconst_rrad(lnRrad,Pstar,epsEnd,potEnd):
    cmbMeasuredQuart = mp.mpf('0.25')*mp.log(Pstar*mp.mpf('8')*mp.pi**2)
    kstar, lnMpcToKappa, HubbleSquareRootOf3OmegaRad, RelatDofRatio = mp.mpf('0.05'), mp.mpf('130.282'), mp.mpf('7.5437e-63'), mp.mpf('1')
    N0 = mp.log(kstar) - lnMpcToKappa - mp.mpf('0.5')*mp.log(HubbleSquareRootOf3OmegaRad) - mp.mpf('0.25')*mp.log(RelatDofRatio)
    return - N0 + cmbMeasuredQuart-mp.mpf('0.25')*mp.log(potEnd/(mp.mpf('3')-epsEnd)) + lnRrad

def x_rrad(alpha,phi0,lnRrad,Pstar):
    xEnd = x_endinf(alpha,phi0)
    epsOneEnd = norm_eps1(xEnd,alpha,phi0)
    potEnd = norm_potential(xEnd,alpha,phi0)
    primEnd = efold_primitive(xEnd,alpha,phi0)
    calF = get_calfconst_rrad(lnRrad,Pstar,epsOneEnd,potEnd)
    calFplusNuEnd = calF+primEnd
    x_eps10 = 1
    return mp.findroot(lambda x : find_x_rrad(x, alpha, phi0,calFplusNuEnd), mp.mpf('1'), solver='halley', verbose = False)

def find_reheat_rrad_leadorder(nuStar,calFplusNuEnd,epsOneStar,Vstar):
    return nuStar - calFplusNuEnd + mp.mpf('0.25')*mp.log(mp.mpf('9')/(epsOneStar*Vstar))

def find_x_rrad(x,alpha,phi0,calFplusNuEnd):
    nuStar = efold_primitive(x,alpha,phi0)
    epsOneStar = norm_eps1(x,alpha,phi0)
    Vstar = norm_potential(x,alpha,phi0)
    res = find_reheat_rrad_leadorder(nuStar,calFplusNuEnd,epsOneStar,Vstar)
    return res

def ns_from_alpha(alpha, phi0B, lnRrad, Pstar):
    phi0 = phi0B*((5*alpha+mp.sqrt(mp.mpf('25')*alpha**2-mp.mpf('9')))/(9*alpha))**(-mp.mpf('0.25'))
    xstar = x_rrad(alpha, phi0, lnRrad, Pstar)
    return mp.mpf('1')-2*norm_eps1(xstar, alpha, phi0)-norm_eps2(xstar, alpha, phi0)

def alpha_coeff(phi0_B,lnRrad):
    phi0_B_Vec=[0.000001,0.000003,0.00001,0.0001,0.001,0.01,0.1]
    if lnRrad == 0:
      coeff_Vec=[2.5,2.2,2.05,1.7,1.4,1.2,1.]
    elif lnRrad == -10:
      coeff_Vec=[5.2,4.4,3.8,3.,2.4,2.,1.6]
    i=0
    while(phi0_B > phi0_B_Vec[i] and i < 5):
        i=i+1
    i = i-1
    return coeff_Vec[i]+(coeff_Vec[i+1]-coeff_Vec[i])*mp.log(phi0_B/phi0_B_Vec[i])/mp.log(phi0_B_Vec[i+1]/phi0_B_Vec[i])

def alpha_from_phi0B_and_ns(phi0B, ns, lnRrad,Pstar):
    alpha_min = mp.mpf('1')-alpha_coeff(phi0B,lnRrad)*phi0B**4*mp.pi**2/(mp.mpf('900')*mp.mpf('50')**2)
    alpha_max = mp.fsub(mp.mpf('1'),mp.mpf('1e-30'))
    return mp.findroot(lambda alpha : ns_from_alpha(alpha, phi0B, lnRrad, Pstar)-ns, (alpha_min, alpha_max), solver='anderson', verbose=False)

def aspic(lnRrad, ns_f, phi0_input, Pstar):
    phi0B_list, phistar_list, alpha_list, mphi_list, A6_list, lambda6_list, ns_list, r_list, alphas_list = [], [], [], [], [], [], [], [], []
    nphi0 = len(phi0_input)
    for i, phi0B in enumerate(phi0_input):
#         start = time.process_time()

        alpha = alpha_from_phi0B_and_ns(phi0B/Mp, ns_f, lnRrad, Pstar)
        phi0 = phi0B*((5*alpha+mp.sqrt(mp.mpf('25')*alpha**2-mp.mpf('9')))/(9*alpha))**(-mp.mpf('0.25'))
        xstar = x_rrad(alpha, phi0/Mp, lnRrad, Pstar)
        eps1 = norm_eps1(xstar,alpha,phi0/Mp)
        eps2 = norm_eps2(xstar,alpha,phi0/Mp)
        eps3 = norm_eps3(xstar,alpha,phi0/Mp)
        
        ns, r, alpha_s = 1 - 2*eps1 - eps2, 16*eps1, -2*eps1*eps2-eps2*eps3
        M = (Pstar * 8 * mp.pi**2 * Mp**2 * eps1 * 3 * Mp**2 / norm_potential(xstar,alpha,phi0/Mp))**mp.mpf('0.25')
        mphiBoehm2 = 2 * M**4 / (phi0)**2 
        AB=mp.sqrt(80*alpha)*M**2/(phi0)
        lambdaB=Mp**3*mp.sqrt(alpha/5)*M**2/(phi0)**5
 
        phi0B_list.append(phi0B)
        phistar_list.append(xstar*phi0)
        alpha_list.append(alpha)
        mphi_list.append(mphiBoehm2**0.5)
        A6_list.append(AB)
        lambda6_list.append(lambdaB)
        ns_list.append(ns)
        r_list.append(r)
        alphas_list.append(alpha_s)

#     print('□', end='')
    return phi0B_list, phistar_list, alpha_list, mphi_list, A6_list, lambda6_list, ns_list, r_list, alphas_list

def DV(V, phi):
     return (V(phi+1)-V(phi-1))/mp.mpf('2')

def D2V(V, phi):
    return (V(phi+1)+V(phi-1)-2*V(phi))/mp.mpf('1')

def eps1_(V, phi):
    return mp.fdiv(mp.power((mp.fmul((mp.mpf(1)*Mp),mp.fdiv(DV(V, phi),V(phi)))),2),2)

def eps2_(V, phi):
    return mp.fmul(mp.fmul(2,mp.power(Mp,2)),mp.fsub(mp.power(mp.fdiv(DV(V, phi),V(phi)),2),mp.fdiv(D2V(V, phi),V(phi))))

def N(V, phimin, phimax):
    mp.dps = 50
    integ = mp.quad(lambda phi : V(phi)/DV(V, phi), [phimin, phimax])
    mp.dps = 500
#     print('.', end="", flush=True)
    return -integ/Mp**2

def find_phi_st(V, phi, phi_end, Pstar, lnRrad):
    kstar, lnMpcToKappa, HubbleSquareRootOf3OmegaRad, RelatDofRatio = 0.05, 130.282, 7.5437e-63, 1
    N0 = log(kstar) - lnMpcToKappa - 0.5*log(HubbleSquareRootOf3OmegaRad) -0.25*log(RelatDofRatio)
    Delta_N_star = N(V, phi, phi_end)
    return -Delta_N_star + lnRrad - N0 - 0.25*mp.log(9/eps1_(V, phi)/(3-eps1_(V, phi_end))*V(phi_end)/V(phi))+0.25*mp.log(8*mp.pi**2*Pstar)

def ns_(V, phi):
    return mp.fsub(mp.fsub(1,2*eps1_(V, phi)),eps2_(V, phi))

def endinf(V, start):
    return mp.findroot(lambda phi: eps1_(V, phi)-1, 0.95*start, tol = 1e-30)

def phi_star(V, Pstar, lnRrad, phi_single):
    # extensions dispos si pas de guess, ou si on explore les potentiels non monotones
    phi_0 = mp.findroot(lambda phi : D2V(V, phi), phi_single, tol=5e-18)
    try:
        phi_0
    except:
        print('échec trouver un phi_0')
        return nan
    phi_start = phi_0
    phi_end = endinf(V, phi_start*0.98)
    Ntot = N(V, phi_end, phi_start-1)

    if -Ntot > 22.4:
#         try:
        phi_star = mp.findroot(lambda phi : find_phi_st(V, phi, phi_end, Pstar, lnRrad), x0 = (phi_start-10,phi_start+10), maxsteps = 30, verbose = False, method='ridder', tol = 1e-25)
        return phi_star
#         except:
#             print('échec trouver un phi_star (malgré assez efolds...)')
#             return mp.mpf(nan)
    else:
        print('pas assez efolds')
        return float(nan)

def P_star(V, phi_sta):
    Vstar = V(phi_sta)
    eps1star = eps1_(V, phi_sta)
    return Vstar/(Mp**4*24*mp.pi**2*eps1star)

def mphi_versus_phi_V_prime_eq_0f(phi, A6, lambda6, mphi_ini, infl_type):
    return mp.findroot(lambda mphi : DV(lambda phi : V_MSSM(phi, infl_type, mphi, A6, lambda6), phi),mphi_ini, verbose=False, tol=5e-18)

def mphi_and_phi0(A6, lambda6, mphi_ini, phi0_tree, infl_type):
    single_point = mp.findroot(lambda phi : DV(lambda phi : mphi_versus_phi_V_prime_eq_0f(phi, A6, lambda6, mphi_ini, infl_type),phi), phi0_tree, verbose = False, tol=1e-25)
#     print('◉', end='')
    return mphi_versus_phi_V_prime_eq_0f(single_point, A6, lambda6, mphi_ini, infl_type), single_point

def ns_star(mphi_start, phi0_start, A6, lambda6, Pstar, lnRrad, infl_type):
    phi_sta = phi_star(lambda phi : V_MSSM(phi, infl_type, mphi_start, A6, lambda6), Pstar, lnRrad, phi0_start)
    ns_star = ns_(lambda phi : V_MSSM(phi, infl_type, mphi_start, A6, lambda6), phi_sta)
    res_P = P_star(lambda phi : V_MSSM(phi, infl_type, mphi_start, A6, lambda6), phi_sta)
    return float(ns_star)

def from_ns_find_mphi(ns, mphi_start, phi0_start, A6, lambda6, Pstar, lnRrad, infl_type):
    ns_star_i = lambda i : ns_star(mphi_start*(mp.mpf('1')+i*mp.mpf('1e-24')), phi0_start, A6, lambda6, Pstar, lnRrad, infl_type)
    i_tuned = mp.findroot(lambda i : ns_star_i(i)-ns, 0, verbose=False,tol = 1e-16)
    mphi_tuned = mphi_start*(mp.mpf('1')+i_tuned*mp.mpf('1e-24'))
    return mphi_tuned

def normalize(mphi_nsed, A6, lambda6, phi0_start, Pstar, lnRrad, infl_type):
    V_to_renormalize =  lambda phi : V_MSSM(phi, infl_type, mphi_nsed, A6, lambda6)
    phi0_to_renormalize = mp.findroot(lambda phi : D2V(V_to_renormalize, phi), phi0_start, tol=5e-18)
    phi_sta_to_renormalize = phi_star(V_to_renormalize, Pstar, lnRrad, phi0_to_renormalize)
    P_star_to_renormalize = P_star(V_to_renormalize, phi_sta_to_renormalize)
    P_star_cosmo = Pstar
    normalization = P_star_cosmo/P_star_to_renormalize
    mphi, A6, lambda6 = mphi_nsed*mp.sqrt(normalization), A6*mp.sqrt(normalization), lambda6*mp.sqrt(normalization)
    ns_star = ns_(lambda phi : V_MSSM(phi, infl_type, mphi, A6, lambda6), phi_sta_to_renormalize)
#     print(' ns = ', nstr(ns_star, 6) ,' As = ',sci(P_star_cosmo))
    return mphi, A6, lambda6 , phi0_to_renormalize, phi_sta_to_renormalize

def rge_point_generator2(A6, lambda6, mphi_tree, phi0_tree, ns_input, Pstar, lnRrad, infl_type):
    start_time = time.process_time()
    mphi, phi0 = mphi_tree, phi0_tree
    mphi_infl, phi0_infl = mphi_and_phi0(A6, lambda6, mphi, phi0, infl_type)
    alpha = aspic(lnRrad, ns_input, [phi0], Pstar)[2][0]
    mphi_loop = mphi_infl/mp.sqrt(alpha)
    mphi_nsed = from_ns_find_mphi(ns_input, mphi_loop, phi0_infl, A6, lambda6, Pstar, lnRrad, infl_type)

#     print('')
    mphi_renormalized, A6_renormalized, lambda6_renormalized, phi0_to_renormalize, phi_sta_to_renormalize = normalize(mphi_nsed, A6, lambda6, phi0_infl, Pstar, lnRrad, infl_type)
    
#     print('\n\nmphi =', nstr(mphi_renormalized, 50))
#     print('A6 =', nstr(A6_renormalized, 50))
#     print('lambda6 =', nstr(lambda6_renormalized, 50))

    V_renormalized =  lambda phi : V_MSSM(phi, infl_type, mphi_renormalized, A6_renormalized, lambda6_renormalized)
    phi0_renormalized = mp.findroot(lambda phi : D2V(V_renormalized, phi), phi0_to_renormalize, tol=1e-17)
    phi_sta_renormalized = phi_sta_to_renormalize
    
    ns_out = ns_(V_renormalized, phi_sta_renormalized)
#     print('\nns =', nstr(ns_out, 20))
    Pstar_out = P_star(V_renormalized, phi_sta_renormalized)
#     print('P_star =', nstr(Pstar_out, 20))
    phi0 = mp.findroot(lambda phi : D2V(V_renormalized, phi), phi_sta_renormalized)
#     print('phi0 =', nstr(phi0,40))
    r = 16*eps1_(V_renormalized, phi_sta_renormalized)
#     print('r =', nstr(r, 20))
#     print('lnRrad =', lnRrad)

    time_step = time.process_time() - start_time
#     print('\n'+str(time_step) +" sec.\n")
    return mphi_renormalized, A6_renormalized, lambda6_renormalized, ns_out, Pstar_out, phi0, r, phi_sta_renormalized

def contour2(phi0_input_list, lnRrad, ns_input, As_input, infl_type):
    compatible_mphi, compatible_A6, compatible_lambda6, compatible_ns, compatible_Ps, compatible_phi0, compatible_r, compatible_phi_star = [], [], [], [], [], [], [], []
    for i, phi0_input in enumerate(phi0_input_list):
#         print('step '+str(i+1)+'/'+str(len(phi0_input_list))+' (scale = '+"%.5e"%phi0_input+' GeV):\n')
        phi0B_list, phistar_list, alpha_list, mphi_list, A6_list, lambda6_list, ns_list, r_list, alphas_list = aspic(lnRrad, ns_input, [phi0_input], As_input)
        A6, lambda6, mphi_tree, phi0_tree = A6_list[0], lambda6_list[0], mphi_list[0], phi0B_list[0]

#         print('')
        
        mphi_renormalized, A6_renormalized, lambda6_renormalized, ns_out, Ps_out, phi0, r, phi_starr = rge_point_generator2(A6, lambda6, mphi_tree, phi0_tree, ns_input, As_input, lnRrad, infl_type)
        
        compatible_mphi.append(mphi_renormalized)
        compatible_A6.append(A6_renormalized)
        compatible_lambda6.append(lambda6_renormalized)
        compatible_ns.append(ns_out)
        compatible_Ps.append(Ps_out)
        compatible_phi0.append(phi0)
        compatible_r.append(r)
        compatible_phi_star.append(phi_starr)
#         print('\n-------------------------------------------------------------------------------------------------------------------------------\n')
        print(end='.')
    return compatible_mphi, compatible_A6, compatible_lambda6, compatible_ns, compatible_Ps, compatible_phi0, compatible_r, compatible_phi_star

def clean(x):
    return str(float(x*mp.mpf('10')**mp.mpf(str(1-int(mp.log10(x)))))).replace('.','')

def name_file(infl_type, lnRrad, As_input, ns_input, special=0):
    if special==0: return infl_type+'_m'+str(abs(int(lnRrad)))+'_'+clean(As_input)+'_0'+clean(ns_input)+'.csv'
    else: return 'tree_'+infl_type+'_m'+str(abs(int(lnRrad)))+'_'+clean(As_input)+'_0'+clean(ns_input)+'.csv'


def make_many_contour(direc, infl_type_list, lnRrad_list, As_input_list, ns_input_list, key_name=''):
    
    """ Si infl_type = tree, construit autant de .csv que de ns As lnRrad donnés, à partir de la liste de phi0. 
        Si infl_type != tree, construit autant de .csv que de ns As lnRrad donnés, à partir de la liste de phi0, 
        et une autre série d'autant de .csv tree, à partir de la liste des phi0 rge légèrement shiftés  
        Nom du fichier output : direc/key_name+infl_type+lnRrad+As+ns.csv
    """
    
    for infl_type in infl_type_list:
        if infl_type == 'tree':
            for lnRrad in lnRrad_list:
                for As_input in As_input_list:
                    for ns_input in ns_input_list:
                        start_time = time.process_time()
                        phi0B_list, phistar_list, alpha_list, mphi_list, A6_list, lambda6_list, ns_list, r_list, alphas_list = aspic(lnRrad, ns_input, phi0_input_list, As_input)
                        data = pd.DataFrame(data={'phi0B':phi0B_list, 'mphi':mphi_list, 'A6':A6_list, 'lambda6':lambda6_list, 'ns':ns_list, 'Ps':[As_input]*len(phi0_input_list), 'r':r_list, 'phi*':phistar_list}, dtype=str)
                        total_name_file = direc+key_name+name_file(infl_type, lnRrad, As_input, ns_input)
                        data.to_csv(total_name_file)
                        time_step = time.process_time() - start_time
                        print(total_name_file+' done in '+str(time_step) +" sec.")
        else:
            for lnRrad in lnRrad_list:
                for As_input in As_input_list:
                    for ns_input in ns_input_list:
                        start_time = time.process_time()
                        compatible_mphi, compatible_A6, compatible_lambda6, compatible_ns, compatible_Ps, compatible_phi0, compatible_r, compatible_phi_star = contour2(phi0_input_list, lnRrad, ns_input, As_input, infl_type)
                        data = pd.DataFrame(data={'phi0B':compatible_phi0, 'mphi':compatible_mphi, 'A6':compatible_A6, 'lambda6':compatible_lambda6, 'ns':compatible_ns, 'Ps':compatible_Ps, 'r':compatible_r, 'phi*':compatible_phi_star}, dtype=str)
                        total_name_file = direc+key_name+name_file(infl_type, lnRrad, As_input, ns_input)
                        data.to_csv(total_name_file)
                        time_step = time.process_time() - start_time
                        print('\n'+total_name_file+' done in '+str(time_step) +" sec.")
                        start_time = time.process_time()

                        phi0B_list, phistar_list, alpha_list, mphi_list, A6_list, lambda6_list, ns_list, r_list, alphas_list = aspic(lnRrad, ns_input, compatible_phi0, As_input)
                        data = pd.DataFrame(data={'phi0B':phi0B_list, 'mphi':mphi_list, 'A6':A6_list, 'lambda6':lambda6_list, 'ns':ns_list, 'Ps':[As_input]*len(phi0_input_list), 'r':r_list, 'phi*':phistar_list}, dtype=str)
                        total_name_file = direc+key_name+name_file(infl_type, lnRrad, As_input, ns_input, 1)
                        data.to_csv(total_name_file)
                        time_step = time.process_time() - start_time
                        print(total_name_file+' done in '+str(time_step) +" sec.")
                        
    return 'Done'



In [2]:
direc='tree_reheat_contours/'
infl_type_list=['tree']
lnRrad_list = [mp.mpf('-10'), mp.mpf('0')]
As_input_list = [mp.mpf('2.0409490229434265e-9'), mp.mpf('2.0989031673191437e-9'), mp.mpf('2.158502959289468e-9')]
ns_input_list = [mp.mpf('0.9530'), mp.mpf('0.9653'), mp.mpf('0.9776')]
phi0_input_list = np.array([mp.mpf(str(x)) for x in np.linspace(7e13, 3e16, 60)])

make_many_contour(direc, infl_type_list, lnRrad_list, As_input_list, ns_input_list)

tree_reheat_contours/tree_m10_20409490229434266_0953.csv done in 31.169923476 sec.
tree_reheat_contours/tree_m10_20409490229434266_09653.csv done in 30.035041796 sec.
tree_reheat_contours/tree_m10_20409490229434266_09776.csv done in 30.247442402999994 sec.
tree_reheat_contours/tree_m10_2098903167319144_0953.csv done in 32.31613372300001 sec.
tree_reheat_contours/tree_m10_2098903167319144_09653.csv done in 31.054546406 sec.
tree_reheat_contours/tree_m10_2098903167319144_09776.csv done in 29.957020133000015 sec.
tree_reheat_contours/tree_m10_2158502959289468_0953.csv done in 30.85140978300001 sec.
tree_reheat_contours/tree_m10_2158502959289468_09653.csv done in 30.888610913999997 sec.
tree_reheat_contours/tree_m10_2158502959289468_09776.csv done in 29.576023245000016 sec.
tree_reheat_contours/tree_m0_20409490229434266_0953.csv done in 31.589295566000033 sec.
tree_reheat_contours/tree_m0_20409490229434266_09653.csv done in 31.329777021999973 sec.
tree_reheat_contours/tree_m0_2040949022943

'Done'

In [3]:
direc='rge_lin_contours/'
# infl_type_list = ['lle', 'udd']
infl_type_list = ['udd']
lnRrad_list = [mp.mpf('0')]
As_input_list = [mp.mpf('2.0989031673191437e-9')]
ns_input_list = [mp.mpf('0.9530'), mp.mpf('0.9653'), mp.mpf('0.9776')]
phi0_input_list = np.array([mp.mpf(str(x)) for x in np.linspace(7e13, 7e16, 20)])

make_many_contour(direc, infl_type_list, lnRrad_list, As_input_list, ns_input_list, 'lin_')

....................
rge_lin_contours/lin_udd_m0_2098903167319144_0953.csv done in 8133.654303913 sec.
rge_lin_contours/lin_tree_udd_m0_2098903167319144_0953.csv done in 83.15425779299949 sec.
....................
rge_lin_contours/lin_udd_m0_2098903167319144_09653.csv done in 8072.640222259999 sec.
rge_lin_contours/lin_tree_udd_m0_2098903167319144_09653.csv done in 75.87567942100031 sec.
....................
rge_lin_contours/lin_udd_m0_2098903167319144_09776.csv done in 7076.907794703999 sec.
rge_lin_contours/lin_tree_udd_m0_2098903167319144_09776.csv done in 77.95082232300047 sec.


'Done'

In [3]:
direc='rge_log_contours/'
infl_type_list = ['lle', 'udd']
lnRrad_list = [mp.mpf('0')]
As_input = [mp.mpf('2.0989031673191437e-9')]
ns_input_list = [mp.mpf('0.9653')]
phi0_input_list = np.array([mp.mpf(str(x)) for x in np.logspace(np.log10(7e13), np.log10(7e16), 30)])

make_many_contour(direc, infl_type_list, lnRrad_list, As_input_list, ns_input_list, 'log_')

..............................
rge_log_contours/log_lle_m0_2098903167319144_09653.csv done in 10716.262227308 sec.
rge_log_contours/log_tree_lle_m0_2098903167319144_09653.csv done in 120.12949086300068 sec.
..............................
rge_log_contours/log_udd_m0_2098903167319144_09653.csv done in 11362.199594631 sec.
rge_log_contours/log_tree_udd_m0_2098903167319144_09653.csv done in 124.44319594600165 sec.


'Done'